# Import the necessary packages 

In [1]:
from datetime import date, timedelta
import warnings


from dwave_qbsolv import QBSolv 
import neal
import numpy as np
import pandas as pd
import pypfopt
import qubovert as qv 
import sympy
import yfinance as yf

warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Notebook with helper functions (not publically available)
from finance_notebook_helper import get_correlation_matrix, get_return_and_risk, create_variables, create_expected_return_and_volatility_expressions, create_objective_function, get_portfolio_information

# Portfolio Risk

Portfolio risk is measured by the standard deviation, $\sigma$. The greater the standard deviation, the greater the risk. Given a portfolio, $P$, with two assets, $A$ and $B$, we represent the weights of the assets in the portfolio with $w_{i}$ (with $i =$ {A,B}), and the corresponding portfolio standard deviation as:

$σ_{_P}$ = ($w_{_A}^{2}σ_{_A}^{2}$ + $w_{_B}^{2}σ_{_B}^{2}$  + $2w_{_A}w_{_B}σ_{_A}σ_{_B}ρ_{_{AB}}$)$^{1/2}$

Given three assets ($A,B,$ and $C$) in $P$, our portfolio standard deviation would be:

$σ_{_P}$ = ($w_{_A}^{2}σ_{_A}^{2}$ + $w_{_B}^{2}σ_{_B}^{2}$ + $w_{_C}^{2}σ_{_C}^{2}$ + $2w_{_A}w_{_B}σ_{_A}σ_{_B}ρ_{_{AB}}$ + $2w_{_B}w_{_C}σ_{_B}σ_{_C}ρ_{_{BC}}$ + $2w_{_A}w_{_C}σ_{_A}σ_{_C}ρ_{_{AC}}$)$^{1/2}$

where $\rho$ is a symmetric matrix that contains the correlation coefficient between an asset $i$ and asset $j$. The prodcut ${\sigma_{a_i}}{\sigma_{a_j}}\rho_{ij}$ = $\text{Cov}_{ij}$ is also called the covariance of the assets $a_{i}$ and $a_{j}$. 

In general, for $N$ assets = {$a_1,a_2,...,a_N$} in a portfolio $P$, the square of the portfolio standard deviation, in other words the variance, is given by the following formula:

\begin{align}
\sigma_{_P}^2 &= \sum_{i=1}^{N} {w_{a_i}}^2{\sigma_{a_i}}^2 + 2\sum_{j=1}^{N}\sum_{i<j}^{N}{w_{a_i}}{w_{a_j}}{\sigma_{a_i}}{\sigma_{a_j}}\rho_{ij} \\
 &= \sum_{i=1}^{N} {w_{a_i}}^2{\sigma_{a_i}}^2 + 2\sum_{j=1}^{N}\sum_{i<j}^{N}{w_{a_i}}{w_{a_j}}\text{Cov}_{ij}
\end{align}

subject to $\sum_{i=1}^{N} w_{i} = 1$ and $0 \leq w_{i}$.  

If however, we wanted to choose a subset, M (with $0 < M \leq N$), from the N assets, our portfolio standard deviation would then be:

$$\sigma_{_P}^2 = \sum_{i=1}^N y_{a_i}w_{{a_i}}^2\sigma_{a_i}^2 + 2\sum_
{i=1}^N \sum_{i<j}^N y_{a_i} y_{a_j} w_{{a_i}}w_{{a_j}}\text{Cov}_{ij}$$

where binary variables, $y_{a_i}$ are introduced to control which assets are included in the portfolio. 

# Sharpe Ratio

A useful measure to consider then is the Sharpe Ratio -- which measures a portfolio's "reward to risk" ratio. To do this, we need to define the portfolio return, $R_{_P}$:

$$ R_{_P} = \sum_{i=1}^N w_{a_i}r_{a_{i}} $$

where $ r $ = {$r_{a_1}, r_{a_2}, ... , r_{a_N}$} is the asset return. Given that, a portfolio manager who is reliant on the Sharpe ratio seeks to find a portfolio where the weights of each asset (i.e. the percent of the portfolio that each asset represents of the portfolio) maximize the ratio. 

The Sharpe ratio is traditionally given of the form:

$$\frac{E[R_{_P}]}{\sigma_{_P}}$$

where $E[R_{_P}]$ is the expectation of $R_{_P}$. However, this is problematic to formulate as a QUBO. Dividing with binary variables means dividing by zero, so we'll seek to minimize, instead, the following expression:

$$\sigma_{_P}^2 - E[R_{_P}]$$

such that $$\sum_{i=1}^{N} y_{a_i} = M$$

We'll get many results via annealing and then classically post-process to find the one that gives the actual highest Sharpe ratio.

Given that understanding we will go through an example Sharpe ratio optimization. As an example dataset, we can use stocks from the S \& P 500 Companies (available on wikipedia) for our portfolio:

In [2]:
pd.options.display.float_format = "{:,.2f}".format
# Get dataset to sample some companies 
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df #show dataset in a dataframe format

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


For demonstrative purposes, we can consider a portfolio of $N=10$ stocks, pre-defined subset of the S \& P 500 dataset for repeatability: 

In [3]:
NUM_STOCKS = 10 
assets = ['NEM', 'KEYS', 'WM', 'CE', 'SYF', 'GIS', 'AAL', 'D', 'APH', 'AMGN'] #list of s & p tickers for assets

Given the selection of assets, we can then get the correlation matrix, $\rho_{ij}$, between the choosen assets: 

In [4]:
corr_matrix_df = get_correlation_matrix(assets)
corr_matrix_df

Loading correlation matrix:


100%|████████████████████████████████████| 10/10 [00:01<00:00,  5.88it/s]


symbol,AAL,AMGN,APH,CE,D,GIS,KEYS,NEM,SYF,WM
symbol,,,,,,,,,,
AAL,1.00,-0.80,-0.62,-0.28,-0.55,-0.72,-0.75,-0.70,-0.04,-0.70
AMGN,-0.80,1.00,0.77,0.41,0.47,0.86,0.80,0.71,0.26,0.79
APH,-0.62,0.77,1.00,0.73,0.40,0.85,0.95,0.66,0.68,0.94
CE,-0.28,0.41,0.73,1.00,0.45,0.36,0.70,0.63,0.86,0.64
D,-0.55,0.47,0.40,0.45,1.00,0.37,0.47,0.64,0.22,0.51
GIS,-0.72,0.86,0.85,0.36,0.37,1.00,0.82,0.60,0.32,0.88
KEYS,-0.75,0.80,0.95,0.70,0.47,0.82,1.00,0.67,0.63,0.95
NEM,-0.70,0.71,0.66,0.63,0.64,0.60,0.67,1.00,0.35,0.63
SYF,-0.04,0.26,0.68,0.86,0.22,0.32,0.63,0.35,1.00,0.58


Next, we get the expected return and the corresponding volatility for our sample of assets for a given time frame (for example, 1 month). They are a put into a dataframe as columns:

In [5]:
ret_and_vol = get_return_and_risk(assets, time_period='1mo')
ret_and_vol

Loading expected returns and risks:


100%|████████████████████████████████████| 10/10 [00:01<00:00,  6.43it/s]


,ret,vol
NEM,0.08,0.33
KEYS,0.27,0.29
WM,0.15,0.19
CE,0.01,0.32
SYF,-0.02,0.42
GIS,0.12,0.19
AAL,-0.21,0.46
D,-0.04,0.19
APH,0.12,0.26
AMGN,0.09,0.24


For formulation as a QUBO, we introduce discretization of our weights:

$$ w_{a_i} = \sum_{j=1}^{N} d_j x_{ij} $$

In [6]:
N = len(assets)
discretization = 9 #parameter used in the discretization process
x = create_variables(assets, discretization)

We then reformulate the expected return and volatility with the binary variables, imposing the weight constraint stated in the beginning: 

In [7]:
expected_return, volatility, weight_constraint = create_expected_return_and_volatility_expressions(assets, x, ret_and_vol, corr_matrix_df)

Creating weight constraint...


100%|██████████████████████████████████| 90/90 [00:00<00:00, 8946.89it/s]


Creating objective function...


100%|████████████████████████████████████| 10/10 [00:00<00:00, 44.36it/s]


For our Sharpe ratio optimization as a QUBO, we create an objective function of the form: 

$$ \text{obj} = kE[R_{_P}] - (1-k)\sigma_{_P}^2 $$

with a hyperparameter, $k$, that is preset and tuned to find the optimal Sharpe ratio.

In [8]:
k = 0.8 
obj = create_objective_function(expected_return, volatility, weight_constraint, k)

Finally, we utilize the functionality in QBSolv and qubovert packages solve optimization problem of our objective function as a QUBO using the simulated annealing technique: 

In [9]:
obj_QUBO = obj.to_qubo() #converts our objective function into a QUBO 
sampler = neal.SimulatedAnnealingSampler() #simulated annealing sampler to solve QUBO 
response = QBSolv().sample_qubo(obj_QUBO.Q, solver=sampler, solver_limit=50)

Based on the solution set obtained from simulated annealing, we update the best Sharpe ratio using the portfolio information,

In [10]:
best_sharpe_ratio = float('-inf') #Set Sharpe ratio at the lowest at the start to compare 
best_portfolio = None
for sol in response: #Loops over the simulated annealing solutions to classically post-process the best Sharpe ratio
    solution = obj.convert_solution(sol)
    portfolio = get_portfolio_information(solution, assets, ret_and_vol, corr_matrix_df)
    if portfolio['sharpe'] > best_sharpe_ratio:
        best_sharpe_ratio = portfolio['sharpe']
        best_portfolio = portfolio

and output the value of the expected return, risk, and value of the Sharpe ratio:

In [11]:
print('The expected return value is:', round(best_portfolio['return'], 2))
print('The portfolio risk is:', round(best_portfolio['risk'], 2))
print('The Sharpe ratio for the best portfolio', round(best_portfolio['sharpe'], 2))

The expected return value is: 0.14
The portfolio risk is: 0.25
The Sharpe ratio for the best portfolio 0.58


Lastly, we can view what the optimal stocks and their corresponding weights are in our portfolio:

In [12]:
fin_portfolio = pd.DataFrame(best_portfolio['weights'],['weights'] )#Output portfolio as a dictionary with asset tickers as keys with corresponding asset weight in portfolio

In [13]:
fin_portfolio

,NEM,KEYS,WM,CE,SYF,GIS,AAL,D,APH,AMGN
weights,0.14,0.31,0.33,0.00,0.19,0.01,0.00,0.00,0.00,0.02


In [14]:
sum(best_portfolio['weights'].values()) #Check to see the weight constraint is statisfied

1.0